In [1]:
%matplotlib inline

(tutorial-topi)=
# TOPI 简介
**作者**: [Ehsan M. Kermani](https://github.com/ehsanmok)

这是一个关于 TVM Operator Inventory（TOPI）的介绍性教程。TOPI 提供了 numpy 风格的通用算子和调度，比 TVM 的抽象程度更高。在本教程中，将看到 TOPI 如何将从 TVM 中编写模板代码中拯救出来。


In [2]:
import tvm
import tvm.testing
from tvm import te
from tvm import topi
import numpy as np

## 基本例子

让我们再来看看行之和的算子（相当于 `B = numpy.sum(A, axis=1)`）为了计算二维 TVM 张量 A 的行之和，应该指定符号算子以及调度，如下所述：

In [3]:
n = te.var("n")
m = te.var("m")
A = te.placeholder((n, m), name="A")
k = te.reduce_axis((0, m), "k")
B = te.compute((n,), lambda i: te.sum(A[i, k], axis=k), name="B")
s = te.create_schedule(B.op)

并以人类可读的格式来检查 IR 代码，我们可以做到

In [4]:
m = tvm.lower(s, [A], simple_mode=True)
m["main"]

PrimFunc([A]) attrs={"from_legacy_te_schedule": (bool)1, "global_symbol": "main", "tir.noalias": (bool)1} {
  allocate B[float32 * n], storage_scope = global
  for (i, 0, n) {
    B[i] = 0f
    for (k, 0, m) {
      B[i] = (B[i] + A[((i*stride) + (k*stride))])
    }
  }
}

然而，对于这样普通的算子，我们不得不自己定义 `reduce` 轴，以及用 `te.compute` 进行显式计算。想象一下，对于更复杂的操作，需要提供多少细节。幸运的是，可以用简单的 `topi.sum` 替换这两行，就像 `numpy.sum` 一样。

In [5]:
C = topi.sum(A, axis=1)
ts = te.create_schedule(C.op)
m = tvm.lower(ts, [A], simple_mode=True)
m["main"]

PrimFunc([A]) attrs={"from_legacy_te_schedule": (bool)1, "global_symbol": "main", "tir.noalias": (bool)1} {
  allocate A_red[float32 * n], storage_scope = global
  for (ax0, 0, n) {
    A_red[ax0] = 0f
    for (k1, 0, m) {
      A_red[ax0] = (A_red[ax0] + A[((ax0*stride) + (k1*stride))])
    }
  }
}

## Numpy 风格的运算符重载

我们可以用 `topi.broadcast_add` 来添加两个张量，它们有正确的（可广播的特定）形状。甚至更短，TOPI 为这种常见的操作提供了运算符重载。比如说：

In [6]:
x, y = 100, 10
a = te.placeholder((x, y, y), name="a")
b = te.placeholder((y, y), name="b")
c = a + b  # same as topi.broadcast_add
d = a * b  # same as topi.broadcast_mul

用同样的语法重载，TOPI 处理将一个原语（`int`, `float`）广播到一个张量 `d - 3.14`。

## 通用的调度和融合操作

到目前为止，我们已经看到了一个例子，说明 TOPI 如何使我们免于在低级别的 API 中编写显式计算。但它并没有在这里停止。我们仍然像以前一样进行调度。TOPI 还提供了更高层次的调度方案，这取决于特定的环境。例如，对于 CUDA，我们可以只用 `topi.sum` 来调度以下一系列以 `topi.generic.schedule_reduce` 结束的操作

In [7]:
e = topi.elemwise_sum([c, d])
f = e / 2.0
g = topi.sum(f)
with tvm.target.cuda():
    sg = topi.cuda.schedule_reduce(g)
    print(tvm.lower(sg, [a, b], simple_mode=True))

@main = primfn(a_1: handle, b_1: handle) -> ()
  attr = {"from_legacy_te_schedule": True, "global_symbol": "main", "tir.noalias": True}
  buffers = {a: Buffer(a_2: Pointer(float32), float32, [10000], []),
             b: Buffer(b_2: Pointer(float32), float32, [100], [])}
  buffer_map = {a_1: a, b_1: b}
  preflattened_buffer_map = {a_1: a_3: Buffer(a_2, float32, [100, 10, 10], []), b_1: b_3: Buffer(b_2, float32, [10, 10], [])} {
  allocate(T_divide_red: Pointer(global float32), float32, [1]), storage_scope = global;
  attr [IterVar(threadIdx.x: int32, [0:1024], "ThreadIndex", "threadIdx.x")] "thread_extent" = 1024;
  allocate(T_divide_red.rf: Pointer(local float32), float32, [1]), storage_scope = local;
  allocate(reduce_temp0: Pointer(local float32), float32, [1]), storage_scope = local {
    T_divide_red.rf_1: Buffer(T_divide_red.rf, float32, [1], [], scope="local", align=4)[0] = 0f32
    for (k0.k1.fused.k2.fused.outer: int32, 0, 10) {
      if @tir.likely((((((k0.k1.fused.k2.fused.o

/media/pc/data/4tb/lxw/anaconda3/envs/tvm-mxnet/lib/python3.10/site-packages/tvm/target/target.py:347: UserWarning: Try specifying cuda arch by adding 'arch=sm_xx' to your target.
  warnings.warn("Try specifying cuda arch by adding 'arch=sm_xx' to your target.")


正如你所看到的，预定的计算阶段已经被积累起来，我们可以通过以下方式检查它们

In [8]:
print(sg.stages)

[stage(a, placeholder(a, 0x55dde093b7f0)), stage(b, placeholder(b, 0x55dddfadec40)), stage(T_add, compute(T_add, body=[(a[ax0, ax1, ax2] + b[ax1, ax2])], axis=[iter_var(ax0, range(min=0, ext=100)), iter_var(ax1, range(min=0, ext=10)), iter_var(ax2, range(min=0, ext=10))], reduce_axis=[], tag=broadcast, attrs={})), stage(T_multiply, compute(T_multiply, body=[(a[ax0, ax1, ax2]*b[ax1, ax2])], axis=[iter_var(ax0, range(min=0, ext=100)), iter_var(ax1, range(min=0, ext=10)), iter_var(ax2, range(min=0, ext=10))], reduce_axis=[], tag=broadcast, attrs={})), stage(T_elemwise_sum, compute(T_elemwise_sum, body=[(T_add[ax0, ax1, ax2] + T_multiply[ax0, ax1, ax2])], axis=[iter_var(ax0, range(min=0, ext=100)), iter_var(ax1, range(min=0, ext=10)), iter_var(ax2, range(min=0, ext=10))], reduce_axis=[], tag=elemwise, attrs={})), stage(T_divide, compute(T_divide, body=[(T_elemwise_sum[ax0, ax1, ax2]/2f)], axis=[iter_var(ax0, range(min=0, ext=100)), iter_var(ax1, range(min=0, ext=10)), iter_var(ax2, range(m

可以通过与 `numpy` 的结果进行比较来测试正确性，如下所示

In [9]:
import mxnet_cudnn
func = tvm.build(sg, [a, b, g], "cuda")
dev = tvm.cuda(0)
a_np = np.random.uniform(size=(x, y, y)).astype(a.dtype)
b_np = np.random.uniform(size=(y, y)).astype(b.dtype)
g_np = np.sum(np.add(a_np + b_np, a_np * b_np) / 2.0)
a_nd = tvm.nd.array(a_np, dev)
b_nd = tvm.nd.array(b_np, dev)
g_nd = tvm.nd.array(np.zeros(g_np.shape, dtype=g_np.dtype), dev)
func(a_nd, b_nd, g_nd)
tvm.testing.assert_allclose(g_nd.numpy(), g_np, rtol=1e-5)

TOPI 还提供常见的神经网络操作，如带有优化调度的 _softmax_

In [10]:
tarray = te.placeholder((512, 512), name="tarray")
softmax_topi = topi.nn.softmax(tarray)
with tvm.target.Target("cuda"):
    sst = topi.cuda.schedule_softmax(softmax_topi)
    print(tvm.lower(sst, [tarray], simple_mode=True))

@main = primfn(tarray_1: handle) -> ()
  attr = {"from_legacy_te_schedule": True, "global_symbol": "main", "tir.noalias": True}
  buffers = {tarray: Buffer(tarray_2: Pointer(float32), float32, [262144], [])}
  buffer_map = {tarray_1: tarray}
  preflattened_buffer_map = {tarray_1: tarray_3: Buffer(tarray_2, float32, [512, 512], [])} {
  allocate(T_softmax_norm: Pointer(global float32x4), float32x4, [65536]), storage_scope = global;
  attr [IterVar(blockIdx.x: int32, (nullptr), "ThreadIndex", "blockIdx.x")] "thread_extent" = 512;
  allocate(normal_reduce_temp0: Pointer(local float32), float32, [1]), storage_scope = local;
  allocate(reduce_temp0: Pointer(local float32), float32, [1]), storage_scope = local;
  allocate(T_softmax_exp: Pointer(warp float32), float32, [512]), storage_scope = warp;
  allocate(normal_reduce_temp0_1: Pointer(local float32), float32, [1]), storage_scope = local;
  allocate(reduce_temp0_1: Pointer(local float32), float32, [1]), storage_scope = local {
    attr [I

## 融合卷积

我们可以将 `topi.nn.conv2d` 和 `topi.nn.relu` 融合在一起。

```{admonition} 注意
:class: alert alert-info

TOPI 函数都是通用函数。它们对不同的后端有不同的实现，以优化性能。对于每个后端，有必要在计算声明和时间表的目标范围内调用它们。TVM 会根据目标信息选择正确的函数来调用。
```

In [11]:
data = te.placeholder((1, 3, 224, 224))
kernel = te.placeholder((10, 3, 5, 5))

with tvm.target.Target("cuda"):
    conv = topi.cuda.conv2d_nchw(data, kernel, 1, 2, 1)
    out = topi.nn.relu(conv)
    sconv = topi.cuda.schedule_conv2d_nchw([out])
    print(tvm.lower(sconv, [data, kernel], simple_mode=True))

Cannot find config for target=cuda -keys=cuda,gpu -arch=sm_75 -max_num_threads=1024 -thread_warp_size=32, workload=('conv2d_nchw.cuda', ('TENSOR', (1, 3, 224, 224), 'float32'), ('TENSOR', (10, 3, 5, 5), 'float32'), 1, 2, 1). A fallback configuration is used, which may bring great performance regression.


@main = primfn(placeholder_2: handle, placeholder_3: handle) -> ()
  attr = {"from_legacy_te_schedule": True, "global_symbol": "main", "tir.noalias": True}
  buffers = {placeholder: Buffer(placeholder_4: Pointer(float32), float32, [150528], []),
             placeholder_1: Buffer(placeholder_5: Pointer(float32), float32, [750], [])}
  buffer_map = {placeholder_2: placeholder, placeholder_3: placeholder_1}
  preflattened_buffer_map = {placeholder_2: placeholder_6: Buffer(placeholder_4, float32, [1, 3, 224, 224], []), placeholder_3: placeholder_7: Buffer(placeholder_5, float32, [10, 3, 5, 5], [])} {
  allocate(compute: Pointer(global float32), float32, [501760]), storage_scope = global;
  attr [IterVar(blockIdx.z: int32, (nullptr), "ThreadIndex", "blockIdx.z")] "thread_extent" = 5;
  allocate(conv2d_nchw: Pointer(local float32), float32, [14]), storage_scope = local;
  allocate(pad_temp.shared: Pointer(shared float32), float32, [112]), storage_scope = shared;
  allocate(placeholder.share

## 总结

在本教程中，我们已经看到

- 如何使用 TOPI API 进行 numpy 风格运算符的普通操作。
- TOPI 如何为上下文的通用调度和运算符融合提供便利，以生成优化的内核代码。